In [1]:
import futhark_printer as fp
import jax
import jax.numpy as jnp

jax.config.update("jax_enable_x64", True)


def f(x, y):
    return x + y


x0 = jnp.ones((), jnp.int32)
y0 = jnp.ones((), jnp.int32)
cj = jax.make_jaxpr(f)(x0, y0)
# print(cj)

fn = fp.jaxpr_to_futhark_ast(cj, name="my_f")
print(fp.print_function(fn))


let my_f (temp0: i32, temp1: i32) =
  let temp2 = (i32.+) temp0 temp1
  in temp2


In [2]:
import subprocess
import futhark_printer

# 1) Generate Futhark code
src = futhark_printer.print_function(fn)

# 2) Write to a temp file
with open("temp.fut", "w") as f:
    f.write(src)
    print("temp.fut")


# If you installed futhark locally:
subprocess.run(["futhark", "c", "-o", "prog", "temp.fut"], check=True)


temp.fut


CompletedProcess(args=['futhark', 'c', '-o', 'prog', 'temp.fut'], returncode=0)